<a href="https://colab.research.google.com/github/hdparmar/AI-Music/blob/main/Latent_Soundings_workshop_RAVE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title README
!yes | pip install --quiet pyfiglet

import pyfiglet
  
resulta = pyfiglet.figlet_format(f"RAVE", font = "isometric1", )
# resultb = pyfiglet.figlet_format(f"TRAIN", font = "isometric2", )
print(resulta)
# print(resultb)
print("\n")
print("\n")
print("RAVE: Realtime Audio Variational autoEncoder | Authors: Antoine Caillon, Acids-IRCAM | https://github.com/acids-ircam/RAVE | https://arxiv.org/abs/2111.05011")
print("Custom collab implementation by Moisés Horta | http://twitter.com/hexorcismos | http://github.com/moiseshorta")
print("\n")
print("Last Modified: 4.October.2022")


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
yes: stdout: Broken pipe
      ___           ___           ___           ___     
     /\  \         /\  \         /\__\         /\  \    
    /::\  \       /::\  \       /:/  /        /::\  \   
   /:/\:\  \     /:/\:\  \     /:/  /        /:/\:\  \  
  /::\~\:\  \   /::\~\:\  \   /:/__/  ___   /::\~\:\  \ 
 /:/\:\ \:\__\ /:/\:\ \:\__\  |:|  | /\__\ /:/\:\ \:\__\
 \/_|::\/:/  / \/__\:\/:/  /  |:|  |/:/  / \:\~\:\ \/__/
    |:|::/  /       \::/  /   |:|__/:/  /   \:\ \:\__\  
    |:|\/__/        /:/  /     \::::/__/     \:\ \/__/  
    |:|  |         /:/  /       ~~~~          \:\__\    
     \|__|         \/__/                       \/__/    





RAVE: Realtime Audio Variational autoEncoder | Authors: Antoine Caillon, Acids-IRCAM | https://github.com/acids-ircam/RAVE | https://arxiv.org/abs/2111.05011
Custom collab implementation by Moisés Horta | http://twitter.com/hexorc

In [2]:
#@title Check GPU

import subprocess
!nvidia-smi


zsh:1: command not found: nvidia-smi


In [ ]:
#@title Anti-disconnect from Colab

import IPython
js_code = '''
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}
setInterval(ClickConnect,60000)
'''
display(IPython.display.Javascript(js_code))

In [ ]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Clone repository & install dependencies

%cd /content/drive/MyDrive
!git clone --quiet --depth 1 https://github.com/acids-ircam/RAVE.git RAVE_COLLAB
print("Cloned RAVE repository")
%cd /content/drive/MyDrive/RAVE_COLLAB
!curl -L https://repo.anaconda.com/miniconda/Miniconda3-py39_4.12.0-Linux-x86_64.sh -o miniconda.sh
!chmod +x miniconda.sh
!sh miniconda.sh -b -p /content/miniconda
!/content/miniconda/bin/pip install --quiet -r requirements.txt
!/content/miniconda/bin/pip install --quiet --upgrade ipython ipykernel
!pip install youtube-dl
print("Succesfully installed requirements")

In [ ]:
#@title [Optional] Download audio dataset from Youtube

!mkdir dataset 
%cd dataset

youtube_link = "https://www.youtube.com/watch?v=h2OuSwK7yn0" #@param {type:"string"}

!youtube-dl -x --audio-format wav --audio-quality 0 $youtube_link 

In [ ]:
#@title [Optional] Apply audio augmentation to your dataset and check total duration

dataset = "/path/to/your/dataset" #@param {type:"string"}
sample_rate = 48000 #@param {type:"integer"}

%cd $dataset

!/content/miniconda/bin/resample --input $dataset --sr $sample_rate --output $dataset/resampled/

%cd $dataset/resampled/

!/content/miniconda/bin/duration



## *Training*

## Configure your training

Here you can define the parameters for your training, also called *run*. This is a collab version of the `cli_helper.py` script.

Input dataset is the path to your audio dataset. 

**PRO TIP**: using the left panel you can navigate to your Drive, find your dataset folder, right click and copy its path ! 

In [ ]:
training_name = "test"  #@param {type:"string"}
input_dataset = "/path/to/audio/dataset.wav"  #@param {type:"string"}
sampling_rate = 48000  #@param {type:"integer"}
multiband_number = 16  #@param {type:"integer"}
n_signal = 65536  #@param {type:"integer"}
size = "default"  #@param ["small", "default", "large"]
prior = 32  #@param {type:"integer"}
fidelity = 99  #@param {type:"slider", min:80, max:99, step:1}
no_latency_mode = False  #@param {type:"boolean"}
regularization_strength = 0.1  #@param {type:"slider", min:0.001, max:1, step:0.001}
validation_every = 2500  #@param {type:"integer"}

training_name = training_name.replace(" ", "_").lower()
no_latency_mode = "true" if no_latency_mode else "false"
# sampling_rate = str(sampling_rate)
# multiband_number = str(multiband_number)
# n_signal = str(n_signal)
# prior = str(prior)
# fidelity = str(fidelity)
# regularization_strength = str(regularization_strength)


In [ ]:
#@title Setup Tensorboard to monitor training

%cd /content/drive/MyDrive/RAVE_COLLAB
%load_ext tensorboard
%tensorboard --logdir . --bind_all 

In [ ]:
#@title Begin training!

%cd /content/drive/MyDrive/RAVE_COLLAB

!/content/miniconda/bin/python train_rave.py \
  -c $size \
  --data-size $multiband_number \
  --no-latency $no_latency_mode \
  --min-kl $regularization_strength \
  --max-kl $regularization_strength \
  --ckpt /content/drive/MyDrive/RAVE_COLLAB/runs/$training_name/rave/ \
  --preprocessed /content/preprocessed/rave \
  --wav $input_dataset \
  --sr $sampling_rate \
  --n-signal $n_signal \
  --val-every $validation_every \
  --name $training_name

In [ ]:
#@title Resume training


checkpoint = "/path/to/rave.ckpt" #@param {type:"string"}

%cd /content/drive/MyDrive/RAVE_COLLAB
!/content/miniconda/bin/python train_rave.py \
  -c $size --data-size $multiband_number \
  --no-latency $no_latency_mode \
  --min-kl $regularization_strength \
  --max-kl $regularization_strength \
  --ckpt $checkpoint \
  --preprocessed /content/preprocessed/rave \
  --wav $input_dataset \
  --sr $sampling_rate \
  --n-signal $n_signal \
  --val-every $validation_every \
  --name $training_name

In [ ]:
#@title Export trained RAVE as torchscript (.ts)

name_of_run = "test" #@param {type:"string"}
# checkpoint = "/path/to/checkpoint.ckpt" #@param {type:"string"}

# !python /content/RAVE/export_rave.py \
#   --name $name_of_run \
#   --run $checkpoint \
#   --cached true \
#   --fidelity 0.95 \
#   --stereo true

checkpoint = "/path/to/rave.ckpt"  #@param {type:"string"}

%cd /content/drive/MyDrive/RAVE_COLLAB

!/content/miniconda/bin/python export_rave.py --run $checkpoint --cached false --name $name_of_run
!/content/miniconda/bin/python export_rave.py --run $checkpoint --cached true --name {name_of_run}_rt

In [ ]:
#@title Begin training Prior

name_of_run = "test" #@param {type:"string"}
audio_dataset = "/path/to/audio/dataset" #@param {type:"string"}
pretrained_rave = "/path/to/pretrained-rave.ts" #@param {type:"string"}
# preprocessed_audio_location = "/content/temp4"

# !python /content/drive/MyDrive/RAVE_repo/update/RAVE/train_prior.py \
#   --name $name_of_run \
#   --wav $audio_dataset \
#   --preprocessed $preprocessed_audio_location \
#   --pretrained-vae $pretrained_rave


%cd /content/drive/MyDrive/RAVE_COLLAB/

!/content/miniconda/bin/python train_prior.py \
  --pretrained-vae rave_{name_of_run}.ts \
  --preprocessed /content/preprocessed/prior \
  --wav $audio_dataset \
  --name $name_of_run \
  --val-every $validation_every

## *Synthesize audio*

In [ ]:
#@title Perform Timbre Transfer on a folder of wavs

checkpoint = "/path/to/checkpoint.ckpt" #@param {type:"string"}
wav_folder = "/path/to/wav/folder" #@param {type:"string"}

%cd /content/drive/MyDrive/RAVE_COLLAB/

!/content/miniconda/bin/python reconstruct.py \
  --ckpt $checkpoint \
  --wav-folder $wav_folder

In [ ]:
#@title Modify the latent dimensions for timbre transfer exploration

exported_rave_ts = "/path/to/pretrained-rave.ts" #@param {type:"string"}
input_audio = "/path/to/audio.wav" #@param {type:"string"}
output_audio = "/content/rave_out.wav" #@param {type:"string"}

#@markdown > Latent Dimension Parameters
d0 = 1.09 #@param {type:"slider", min:-3, max:3, step:0.01}
d1 = 0 #@param {type:"slider", min:-3, max:3, step:0.01}
d2 = 0 #@param {type:"slider", min:-3, max:3, step:0.01}
d3 = 0 #@param {type:"slider", min:-3, max:3, step:0.01}

import torch
import librosa as li
import soundfile as sf
import IPython

torch.set_grad_enabled(False)

model = torch.jit.load(exported_rave_ts).eval()
print("Loaded",exported_rave_ts)
x = li.load(input_audio)[0]

#ENCODE AUDIO TO LATENT REPRESENTATION
x = torch.from_numpy(x).reshape(1, 1, -1)
z = model.encode(x)
print("Encoded",input_audio)

#BIAS LATENT DIMENSIONS
z[:, 0] += torch.linspace(d0,d0, z.shape[-1])
z[:, 1] += torch.linspace(d1,d1, z.shape[-1])
z[:, 2] += torch.linspace(d2,d2, z.shape[-1])
z[:, 3] += torch.linspace(d3,d3, z.shape[-1])

#SYNTHESIZE AUDIO
y = model.decode(z).numpy().reshape(-1)
sf.write(output_audio, y, 48000)

print("RAVE output:")
IPython.display.Audio(output_audio)


In [ ]:
#@title Unconditional neural audio synthesis using RAVE + Prior

prior_checkpoint = "/path/to/prior.ckpt" #@param {type:"string"}
exported_rave = "/path/to/pretrained-rave.ts" #@param {type:"string"}
seconds_to_generate = 15 #@param {type:"integer"}
output_audio = "/content/rave+prior_output.wav" #@param {type:"string"}

%cd /content/drive/MyDrive/RAVE_COLLAB/


!/content/miniconda/bin/python generation.py \
  --prior-ckpt $prior_checkpoint \
  --exported-rave $exported_rave \
  --length $seconds_to_generate \
  --out-path $output_audio

print("RAVE+Prior output:")
IPython.display.Audio(output_audio)